Pytania:
- Czy i jak oszacowywac czas przesiadki? w ktorym z algorytmow?
- Czy zakres wartosci heurystyki wplywa na wybierane sciezki? Jak to przeskalowac do wspolnej skali 

musza byc w takiej samej jednostce
np. srednia predkosc jakos 

- Jak wybierac koncowe i poczatkowe przystanki bo jak jest wiele mozliwych na poczatku i na koncu to komplikuje algorytm

jak uwzglednic przesiadke z innych 

roboczo dojscie piechota - np. dojscie minute, nie trzeba obliczac jak dlugo dystans 

heurystyka nie musi miec sensu, modyfikacja ktora musi usprawnic ale nie musi

raczej bedzie gorszy, nie bedzie raczej lepsze, 



In [1]:
import pandas as pd 
pd.options.mode.chained_assignment = None
from pathlib import Path 
import numpy as np
from typing import Union 
from datetime import datetime, timedelta
from queue import PriorityQueue
import re
import geopy.distance
from timeit import default_timer as timer

DATA_DIR = Path('../data')

In [2]:
connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv', 
                               usecols=['line', 'departure_time', 'arrival_time', 'start_stop',
       'end_stop', 'start_stop_lat', 'start_stop_lon', 'end_stop_lat',
       'end_stop_lon'])

C:\Users\julia\AppData\Local\Temp\ipykernel_9516\2815982893.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


In [3]:
connection_graph.dtypes

line               object
departure_time     object
arrival_time       object
start_stop         object
end_stop           object
start_stop_lat    float64
start_stop_lon    float64
end_stop_lat      float64
end_stop_lon      float64
dtype: object

In [4]:
connection_graph.columns

Index(['line', 'departure_time', 'arrival_time', 'start_stop', 'end_stop',
       'start_stop_lat', 'start_stop_lon', 'end_stop_lat', 'end_stop_lon'],
      dtype='object')

In [5]:
connection_graph.head()

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon
0,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539
1,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735
2,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376
3,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617
4,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383


In [6]:
len(connection_graph)

996520

In [7]:
connection_graph = connection_graph.drop_duplicates()

In [8]:
len(connection_graph)

456019

Obserwacja: niektóre linie są wyrażone jako int, a niektóre jako str - najlepiej to sprowadzić to wspólnego formatu str

In [9]:
connection_graph['line'].unique()

array(['A', 'D', 'K', 'N', '1', '2', 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
       13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 136, 137, 138, 142, 143, 144, 145, 146, 147,
       148, 149, 150, 151, 152, 206, 240, 241, 242, 243, 244, 245, 246,
       247, 248, 249, 250, 251, 253, 255, 257, 259, 310, 315, 319, 345,
       602, 607, 612, 714, 903, 904, 907, 908, 909, 911, 913, 914, 917,
       920, 921, 923, 924, 927, 930, 931, 933, 934, 936, 937, 938, 941,
       947, 948, 958, 967], dtype=object)

In [10]:
connection_graph['line'] = connection_graph['line'].astype(str)

Założenie: czas przejazdu między przystankami dla tej samej linii może być różny między różnymi godzinami 

In [11]:
krzyki_sowia_conn = connection_graph[(connection_graph['start_stop'] == 'KRZYKI') & (connection_graph['end_stop'] == 'Sowia')]

In [12]:
krzyki_sowia_conn['line'].unique()

array(['A', '126', '248', '251'], dtype=object)

In [13]:
duplicated_stops = connection_graph[connection_graph['start_stop'] == connection_graph['end_stop']]
duplicated_stops

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon
14127,D,08:43:00,08:44:00,Pomorska,Pomorska,51.117895,17.030893,51.117267,17.032001
15247,D,09:25:00,09:26:00,Pomorska,Pomorska,51.117895,17.030893,51.117267,17.032001
15934,D,20:49:00,20:50:00,Pomorska,Pomorska,51.117339,17.031928,51.118012,17.030745
16934,D,20:12:00,20:13:00,Pomorska,Pomorska,51.117339,17.031928,51.118012,17.030745
21374,D,05:46:00,05:47:00,Pomorska,Pomorska,51.117895,17.030893,51.117267,17.032001
...,...,...,...,...,...,...,...,...,...
995006,947,16:25:00,16:26:00,OPORÓW,OPORÓW,51.084150,16.971898,51.083929,16.970916
995033,947,18:25:00,18:26:00,OPORÓW,OPORÓW,51.084150,16.971898,51.083929,16.970916
995051,947,20:25:00,20:26:00,OPORÓW,OPORÓW,51.084150,16.971898,51.083929,16.970916
995078,947,05:05:00,05:06:00,OPORÓW,OPORÓW,51.084150,16.971898,51.083929,16.970916


In [14]:
duplicated_stops[duplicated_stops['start_stop'] == 'Pomorska'].loc[:, ['start_stop', 'start_stop_lat', 'start_stop_lon']].drop_duplicates()

,start_stop,start_stop_lat,start_stop_lon
14127,Pomorska,51.117895,17.030893
15934,Pomorska,51.117339,17.031928
300021,Pomorska,51.117267,17.031899
300086,Pomorska,51.117892,17.031002


POKAZAC ZE DLA TAKIEGO PRZYPADKU BEDZIE DZIALALO

W przypadku implementacji Dijkstry to, że istnieją połączenia między tym samym przystankiem nie będzie miało znaczenia, tj. nie będziemy takich połączeń rozważać

In [15]:
connection_graph.loc[duplicated_stops.iloc[0].name - 5:duplicated_stops.iloc[0].name + 5]

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon
14122,D,08:34:00,08:35:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376
14123,D,08:35:00,08:36:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.137180,17.029450
14124,D,08:36:00,08:39:00,Bałtycka,Kleczkowska,51.137180,17.029450,51.126714,17.027626
14125,D,08:39:00,08:40:00,Kleczkowska,pl. Strzelecki,51.126714,17.027626,51.122708,17.026304
14126,D,08:40:00,08:43:00,pl. Strzelecki,Pomorska,51.122708,17.026304,51.117895,17.030893
14127,D,08:43:00,08:44:00,Pomorska,Pomorska,51.117895,17.030893,51.117267,17.032001
14128,D,08:44:00,08:45:00,Pomorska,Mosty Pomorskie,51.117267,17.032001,51.114754,17.030627
14129,D,08:45:00,08:47:00,Mosty Pomorskie,Rynek,51.114754,17.030627,51.111912,17.027184
14130,D,08:47:00,08:49:00,Rynek,Narodowe Forum Muzyki,51.111912,17.027184,51.107899,17.026294
14131,D,08:49:00,08:51:00,Narodowe Forum Muzyki,Renoma,51.107899,17.026294,51.104286,17.030093


Heurystyka będzie brała pod uwagę odległość w metrach między przystankami. 

Czy trzeba koniecznie tą odległość manhattan / euklidesową?

In [16]:
stop1 = connection_graph.loc[995006]
geopy.distance.geodesic((stop1['start_stop_lat'], stop1['start_stop_lon']), 
                        (stop1['end_stop_lat'], stop1['end_stop_lon'])).m

73.09182397423561

In [17]:
stop1 = connection_graph.loc[14127]

In [18]:
def get_stop_coords(stop: str):
    return connection_graph[connection_graph['end_stop'] == stop].loc[:, ['end_stop', 'end_stop_lat', 'end_stop_lon']].drop_duplicates()

def compute_stop_coords(stop: str):
    stops = get_stop_coords(stop)
    return stops[['end_stop_lat', 'end_stop_lon']].mean()

In [19]:
get_stop_coords('Pomorska')

,end_stop,end_stop_lat,end_stop_lon
14126,Pomorska,51.117895,17.030893
14127,Pomorska,51.117267,17.032001
15933,Pomorska,51.117339,17.031928
15934,Pomorska,51.118012,17.030745
105438,Pomorska,51.117892,17.031002
239675,Pomorska,51.118058,17.030560
300020,Pomorska,51.117267,17.031899
300086,Pomorska,51.117234,17.031756


In [20]:
goal_stop_coords = compute_stop_coords('Pomorska')

A geodesic is the shortest path between two points on a curved surface, analogous to a straight line on a plane surface

In [21]:
def heuristic(curr_conn, goal_stop_coords):
    return round(geopy.distance.geodesic((curr_conn['end_stop_lat'], curr_conn['end_stop_lon']), 
                                   (goal_stop_coords['end_stop_lat'], goal_stop_coords['end_stop_lon'])).m, 2) 

heuristic(connection_graph.loc[14122], goal_stop_coords)

2663.87

Problem dotyczący dat:
- format nie jest jednolity, tj. istnieją godziny 25
- porównywanie godzin nie daje żądanych wyników, tj. chcielibyśmy, by godzina 00:00 była po 23:59

In [22]:
(datetime.strptime('00:00:00', '%H:%M:%S') > datetime.strptime('23:59:00', '%H:%M:%S'))

False

In [23]:
def separate_time(time):
    return [int(tp) for tp in time.split(':')]

In [24]:
separate_time('20:00:00')

[20, 0, 0]

In [25]:
connection_graph[connection_graph['departure_time'].apply(lambda x: separate_time(x)[0]) >= 24].head()

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon
30969,K,24:01:00,24:03:00,Broniewskiego,Bałtycka,51.135904,17.035327,51.138282,17.029374
30970,K,24:03:00,24:04:00,Bałtycka,Bezpieczna,51.138282,17.029374,51.142459,17.025678
30971,K,24:04:00,24:05:00,Bezpieczna,Paprotna,51.142459,17.025678,51.148181,17.020627
30972,K,24:05:00,24:06:00,Paprotna,Zajezdnia Obornicka,51.148181,17.020627,51.148737,17.021069
31418,K,24:00:00,24:03:00,DWORZEC GŁÓWNY,GALERIA DOMINIKAŃSKA,51.099602,17.035400,51.108019,17.037383


In [26]:
connection_graph[connection_graph['departure_time'].apply(lambda x: separate_time(x)[0]) >= 24].max()

line                      K
departure_time     30:03:00
arrival_time       30:05:00
start_stop          Żmudzka
end_stop            Żmudzka
start_stop_lat    51.196642
start_stop_lon    17.165968
end_stop_lat         51.197
end_stop_lon      17.165968
dtype: object

In [27]:
connection_graph[connection_graph['departure_time'].apply(lambda x: separate_time(x)[0]) <= 4].min()

line                      1
departure_time     03:34:00
arrival_time       03:36:00
start_stop           8 Maja
end_stop             8 Maja
start_stop_lat    50.980426
start_stop_lon    16.704534
end_stop_lat      50.984631
end_stop_lon      16.696798
dtype: object

1. Czas zostanie znormalizowany tak, by wszystkie godziny były w przedziale [0:00; 23:59]

In [28]:
def to_seconds(time: str) -> int:
    h, m, s = separate_time(time)
    return int(h * 3600 + m * 60 + s)

def time_to_normalized_sec(time: str) -> int:
    return to_seconds(time) % (3600 * 24)

In [29]:
connection_graph['departure_sec'] = connection_graph['departure_time'].apply(time_to_normalized_sec)
connection_graph['arrival_sec'] = connection_graph['arrival_time'].apply(time_to_normalized_sec)

In [30]:
connection_graph['arrival_sec']

0         75180
1         75240
2         75300
3         75420
4         75540
          ...  
996209    67140
996210    67260
996211    67320
996212    67380
996213    67500
Name: arrival_sec, Length: 456019, dtype: int64

In [31]:
print(connection_graph['departure_sec'].max() <= (24 * 3600))
print(connection_graph['arrival_sec'].max() <= (24 * 3600))

True
True


2. Stworzymy funkcję, która będzie respektować porządek godzinowy, nie liczbowy

In [32]:
def diff(ts: Union[pd.Series, int], td: int) -> int:
    '''Function that returns difference between ts and td times expressed as seconds 
        - note that this will never be a negative value'''
    d = ts - td 
    if isinstance(ts, pd.Series):
        d[d < 0] += 24 * 3600
    elif d < 0:
        d += 24 * 3600
    return d 

In [33]:
t_01_00 = 1 * 3600
t_22_00 = 22 * 3600
t_23_00 = 23 * 3600
t_00_01 = 0 + 60
t_21_00 = 21 * 3600
t_12_00 = 12 * 3600

print('01:00 - 22:00 = ', timedelta(seconds=diff(t_01_00, t_22_00)))
print('01:00 - 12:00 = ', timedelta(seconds=diff(t_01_00, t_12_00)))
print('23:00 - 22:00 = ', timedelta(seconds=diff(t_23_00, t_22_00)))
print('00:01 - 22:00 = ', timedelta(seconds=diff(t_00_01, t_22_00)))
print('23:00 - 12:00 = ', timedelta(seconds=diff(t_23_00, t_12_00)))
print('12:00 - 22:00 = ', timedelta(seconds=diff(t_12_00, t_22_00)))


01:00 - 22:00 =  3:00:00
01:00 - 12:00 =  13:00:00
23:00 - 22:00 =  1:00:00
00:01 - 22:00 =  2:01:00
23:00 - 12:00 =  11:00:00
12:00 - 22:00 =  14:00:00


In [34]:
t_20_00 = to_seconds('20:00:00')
t_20_01 = to_seconds('20:01:00')
t_20_02 = to_seconds('20:02:00')
t_19_59 = to_seconds('19:59:00')
t_20_03 = to_seconds('20:03:00')

print('19:59:00 - 20:00:00 = ', timedelta(seconds=diff(t_19_59, t_20_00)))
print('20:01:00 - 20:00:00 = ', timedelta(seconds=diff(t_20_01, t_20_00)))
print('20:02:00 - 20:00:00 = ', timedelta(seconds=diff(t_20_02, t_20_00)))
print('20:03:00 - 20:00:00 = ', timedelta(seconds=diff(t_20_03, t_20_00)))

19:59:00 - 20:00:00 =  23:59:00
20:01:00 - 20:00:00 =  0:01:00
20:02:00 - 20:00:00 =  0:02:00
20:03:00 - 20:00:00 =  0:03:00


In [35]:
krzyki_sowia_conn = connection_graph[(connection_graph['start_stop'] == 'KRZYKI') & (connection_graph['end_stop'] == 'Sowia')]
krzyki_sowia_conn.head()

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_sec,arrival_sec
44,A,22:19:00,22:20:00,KRZYKI,Sowia,51.074884,17.006569,51.073793,17.001845,80340,80400
169,A,22:00:00,22:01:00,KRZYKI,Sowia,51.074884,17.006569,51.073793,17.001845,79200,79260
241,A,06:38:00,06:39:00,KRZYKI,Sowia,51.074884,17.006569,51.073793,17.001845,23880,23940
305,A,09:08:00,09:09:00,KRZYKI,Sowia,51.074884,17.006569,51.073793,17.001845,32880,32940
369,A,11:25:00,11:26:00,KRZYKI,Sowia,51.074884,17.006569,51.073793,17.001845,41100,41160


Przykład: dostajemy prawidłowe, logiczne sortowanie wg. godzin odjazdu

In [36]:
sorted_by_dep = krzyki_sowia_conn.sort_values(by='departure_sec')

In [37]:
sorted_by_dep

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_sec,arrival_sec
911433,248,24:13:00,24:14:00,KRZYKI,Sowia,51.075272,17.006664,51.073793,17.001845,780,840
916404,251,24:23:00,24:24:00,KRZYKI,Sowia,51.074884,17.006569,51.073793,17.001845,1380,1440
911275,248,25:13:00,25:14:00,KRZYKI,Sowia,51.075272,17.006664,51.073793,17.001845,4380,4440
916115,251,25:23:00,25:24:00,KRZYKI,Sowia,51.074884,17.006569,51.073793,17.001845,4980,5040
911504,248,26:13:00,26:14:00,KRZYKI,Sowia,51.075272,17.006664,51.073793,17.001845,7980,8040
...,...,...,...,...,...,...,...,...,...,...,...
641864,126,22:18:00,22:19:00,KRZYKI,Sowia,51.075272,17.006664,51.073793,17.001845,80280,80340
44,A,22:19:00,22:20:00,KRZYKI,Sowia,51.074884,17.006569,51.073793,17.001845,80340,80400
2047,A,22:34:00,22:35:00,KRZYKI,Sowia,51.074884,17.006569,51.073793,17.001845,81240,81300
644431,126,22:46:00,22:47:00,KRZYKI,Sowia,51.075272,17.006664,51.073793,17.001845,81960,82020


Warto zauważyć, co może jest oczywsite, że do danego przystanku możemy się dostać wieloma liniami. Będzie to miało znaczenie takie, że mając do wyboru wiele linii do następnego przystanku możemy wybrać tą, którą najszybciej się dostaniemy (tj. arrival_time będzie najmniejszy), ponieważ zawsze możemy się przesiąść - więc nawet jeśli wybierzemy linię, która nie prowadzi nas do celu bezpośrednio, to możemy się na dowolnym przystanku przesiąść w prawidłową linię. 

In [38]:
connection_graph[connection_graph['start_stop'] == 'KRZYKI'][['line', 'end_stop']].drop_duplicates()

,line,end_stop
44,A,Sowia
14074,D,Zimowa
14153,D,Orla
64771,2,Orla
127704,6,Orla
139301,7,Radio i Telewizja
139311,7,Zajezdnia BOREK
139375,7,Orla
246676,14,Orla
283710,17,Radio i Telewizja


In [39]:
end_stops = connection_graph[(connection_graph['start_stop'] == 'KRZYKI')]['end_stop'].unique()

In [40]:
from_sliczna_143 = connection_graph[(connection_graph['start_stop'] == 'Śliczna') & (connection_graph['end_stop'] == 'Uniwersytet Ekonomiczny') & (connection_graph['line'] == '143')]

In [41]:
from_sliczna_143.sort_values(by='departure_sec')

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_sec,arrival_sec
802911,143,05:12:00,05:15:00,Śliczna,Uniwersytet Ekonomiczny,51.085705,17.035382,51.089768,17.026732,18720,18900
795984,143,05:13:00,05:16:00,Śliczna,Uniwersytet Ekonomiczny,51.085705,17.035382,51.089768,17.026732,18780,18960
805822,143,05:34:00,05:37:00,Śliczna,Uniwersytet Ekonomiczny,51.085705,17.035382,51.089768,17.026732,20040,20220
797068,143,05:43:00,05:46:00,Śliczna,Uniwersytet Ekonomiczny,51.085705,17.035382,51.089768,17.026732,20580,20760
802361,143,05:52:00,05:55:00,Śliczna,Uniwersytet Ekonomiczny,51.085705,17.035382,51.089768,17.026732,21120,21300
...,...,...,...,...,...,...,...,...,...,...,...
795308,143,21:05:00,21:08:00,Śliczna,Uniwersytet Ekonomiczny,51.085705,17.035382,51.089768,17.026732,75900,76080
795907,143,21:35:00,21:38:00,Śliczna,Uniwersytet Ekonomiczny,51.085705,17.035382,51.089768,17.026732,77700,77880
796912,143,22:05:00,22:08:00,Śliczna,Uniwersytet Ekonomiczny,51.085705,17.035382,51.089768,17.026732,79500,79680
798266,143,22:35:00,22:38:00,Śliczna,Uniwersytet Ekonomiczny,51.085705,17.035382,51.089768,17.026732,81300,81480


In [42]:
type(end_stops)

numpy.ndarray

In [43]:
connection_graph

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_sec,arrival_sec
0,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539,75120,75180
1,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735,75180,75240
2,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376,75240,75300
3,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617,75300,75420
4,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383,75420,75540
...,...,...,...,...,...,...,...,...,...,...,...
996209,967,18:38:00,18:39:00,Smolec - Lipowa/pętla,Smolec - Dębowa (sklep),51.075005,16.884421,51.076932,16.881543,67080,67140
996210,967,18:39:00,18:41:00,Smolec - Dębowa (sklep),Krzeptów - skrzy.,51.076932,16.881543,51.088752,16.875808,67140,67260
996211,967,18:41:00,18:42:00,Krzeptów - skrzy.,Krzeptów - Boisko,51.088752,16.875808,51.090678,16.885584,67260,67320
996212,967,18:42:00,18:43:00,Krzeptów - Boisko,Krzeptów - Dolina Krzeptowa,51.090678,16.885584,51.092268,16.892523,67320,67380


In [273]:
class Graph:

    def __init__(self, conn_graph, add_change_time):
        self.conn_graph = conn_graph
        self.change_time_compute = add_change_time
        self.stops = None
        self.transform()

    def transform(self):
        '''Applies time transformation functions to departure and arrival times - the results are available in new columns'''
        self.conn_graph = self.conn_graph.drop_duplicates()
        self.conn_graph.loc[:, 'line'] = self.conn_graph.loc[:, 'line'].astype(str)
        self.conn_graph.loc[:, 'departure_sec'] = self.conn_graph['departure_time'].apply(time_to_normalized_sec)
        self.conn_graph.loc[:, 'arrival_sec'] = self.conn_graph['arrival_time'].apply(time_to_normalized_sec)

    def get_start_idx(self):
        return -1

    def add_conn(self, dep_time: int, conn: pd.Series, index: int):
         self.conn_graph.loc[index] = pd.Series({'start_stop': conn['stop'], 'end_stop': conn['stop'], 'departure_sec': dep_time, 'arrival_sec': dep_time,
                                                               'start_stop_lat': conn['stop_lat'], 'start_stop_lon': conn['stop_lon'],
                                                               'end_stop_lat': conn['stop_lat'], 'end_stop_lon': conn['stop_lon']})

    def get_possible_stops(self, stop: str):
        return self.rename_stop(self.conn_graph[self.conn_graph['end_stop'] == stop].loc[:, ['end_stop', 'end_stop_lat', 'end_stop_lon']]).drop_duplicates()

    # We could take as the initial stop the closest stop to the goal stop
    def compute_stop_coords(self, stop: str):
        stops = self.get_possible_stops(stop)
        return stops.iloc[0][['stop_lat', 'stop_lon']]
    
    def get_end_stop(self, stop: str):
        coords = self.compute_stop_coords(stop)
        return pd.Series({'end_stop': stop, 'end_stop_lat': coords['end_stop_lat'], 'end_stop_lon': coords['end_stop_lon']})

    def get_nodes(self):
        '''should we also add end_stop?'''
        return self.conn_graph.loc[:, ['start_stop', 'start_stop_lat', 'start_stop_lon']].drop_duplicates()
    
    def get_lines(self, start_stop, end_stop):
        '''Returns lines connecting two stops'''
        return self.conn_graph[(self.conn_graph['start_stop'] == start_stop) 
                               & (self.conn_graph['end_stop'] == end_stop)]['line'].unique()
    
    def get_stop_names(self):
        '''? this may not make sense because sometimes the routes are different'''
        return self.conn_graph['start_stop'].unique()
    
    def stop_as_tuple(self, stop):
        return (stop['stop'], stop['stop_lat'], stop['stop_lon'])
    
    def is_start_equal_to(self, stop: pd.Series) -> pd.Series:
        return (self.conn_graph['start_stop'] == stop['stop']) & (self.conn_graph['start_stop_lat'] == stop['stop_lat']) & (self.conn_graph['start_stop_lon'] == stop['stop_lon'])

    def is_end_equal_to(self, stop: pd.Series) -> pd.Series:
        return (self.conn_graph['end_stop'] == stop['stop']) & (self.conn_graph['end_stop_lat'] == stop['stop_lat']) & (self.conn_graph['end_stop_lon'] == stop['stop_lon'])

    def rename_stop(self, stop):
        if isinstance(stop, pd.DataFrame):
            return stop.rename({'end_stop_lat': 'stop_lat', 
                                        'end_stop_lon': 'stop_lon', 'end_stop': 'stop'}, axis=1)
        else: 
            return stop.rename({'end_stop_lat': 'stop_lat', 
                                        'end_stop_lon': 'stop_lon', 'end_stop': 'stop'})

    def get_neighbour_stops(self, prev_conn: pd.Series) -> pd.Series:
        '''Returns neighbouring end stops'''
        # return self.conn_graph[self.conn_graph['start_stop'] == start_stop][['line', 'end_stop']].drop_duplicates()
        return self.rename_stop(self.conn_graph[self.is_start_equal_to(prev_conn)][['end_stop', 'end_stop_lat', 'end_stop_lon']]).drop_duplicates()

    

    def get_conn_valid_time_arrivals(self, dep_time: int, prev_stop: pd.Series, next_stop: pd.Series) -> pd.Series:
        conn = self.conn_graph[self.is_start_equal_to(prev_stop) 
                               & self.is_end_equal_to(next_stop)]
 
        time_arrv_diff = diff(conn['arrival_sec'], dep_time)
        time_dep_diff = diff(self.change_time_compute(conn, prev_stop), dep_time)
        
        differences = (time_arrv_diff - time_dep_diff) >= 0
        valid_time_arrv_diff = time_arrv_diff[differences]

        return conn, valid_time_arrv_diff

    def get_sorted_conns(self, dep_time: int, start_stop: pd.Series, end_stop: pd.Series):
        '''get all connections sorted by time arrival'''
        conn, valid_time_arrv_diff = self.get_conn_valid_time_arrivals(dep_time, start_stop, end_stop)

        return conn.loc[valid_time_arrv_diff.sort_values().index]

    def get_earliest_conn(self, dep_time: int, start_stop: pd.Series, end_stop: pd.Series) -> pd.Series:
        '''Returns the earliest connection between two stops'''
        conn, valid_time_arrv_diff = self.get_conn_valid_time_arrivals(dep_time, start_stop, end_stop)

        return conn.loc[valid_time_arrv_diff.idxmin()] if len(valid_time_arrv_diff) > 0 else pd.Series()
    
    def get_earliest_from(self, dep_time: int, start_stop: pd.Series):
        '''Returns all earliest connections to all neighbouring stops'''
        start_stop = self.rename_stop(start_stop)
        possible_conns = [self.get_earliest_conn(dep_time, start_stop, end_stop) for 
                (i, end_stop) in self.get_neighbour_stops(start_stop).iterrows()]
        return [conn for conn in possible_conns if len(conn) > 0]

    def cost_getting_to(self, dep_time: int, next_idx: int, curr_idx: int = None) -> int:
        cost = diff(self.conn_graph.loc[next_idx, 'arrival_sec'], dep_time)
        return cost
    
    def conn_at_index(self, index: int) -> pd.Series:
        return self.conn_graph.loc[index] 

In [241]:
def add_constant_change_time(conns, stop):
    line_cont = conns['line'] == stop['line']
    change_times = pd.Series(data=conns['departure_sec'])
    change_times.loc[line_cont] = (change_times + 60) % (24 * 3600)
    return change_times

In [244]:
connection_graph.loc[65256:65258]

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_sec,arrival_sec
65256,2,23:59:00,24:00:00,Na Szańcach,Jedności Narodowej,51.118902,17.042041,51.121020,17.043260,86340,0
65257,2,24:00:00,24:02:00,Jedności Narodowej,Słowiańska,51.121020,17.043260,51.124070,17.042823,0,120
65258,2,24:02:00,24:03:00,Słowiańska,Zajezdnia OŁBIN,51.124070,17.042823,51.123839,17.041044,120,180


In [243]:
add_constant_change_time(connection_graph.loc[65256:65258], connection_graph.loc[1])

65256    86340
65257        0
65258      120
Name: departure_sec, dtype: int64

In [246]:
g = Graph(connection_graph, add_constant_change_time)

In [139]:
pl_grunw_stop = g.rename_stop(g.conn_at_index(71821))

In [187]:
pl_grunw_stops = g.get_possible_stops('PL. GRUNWALDZKI')

In [188]:
pl_grunw_stops

,stop,stop_lat,stop_lon
14026,PL. GRUNWALDZKI,51.111621,17.060086
14160,PL. GRUNWALDZKI,51.111700,17.060432
53998,PL. GRUNWALDZKI,51.111141,17.061193
54014,PL. GRUNWALDZKI,51.111157,17.061465
64755,PL. GRUNWALDZKI,51.111623,17.059965
64785,PL. GRUNWALDZKI,51.111452,17.060529
97769,PL. GRUNWALDZKI,51.111444,17.059960
217286,PL. GRUNWALDZKI,51.111611,17.060434
507196,PL. GRUNWALDZKI,51.112459,17.060828
548037,PL. GRUNWALDZKI,51.111438,17.060353


In [161]:
g.conn_at_index(14026)

line                            D
departure_time           22:57:00
arrival_time             23:01:00
start_stop         Kochanowskiego
end_stop          PL. GRUNWALDZKI
start_stop_lat          51.115018
start_stop_lon          17.073334
end_stop_lat            51.111621
end_stop_lon            17.060086
departure_sec               82620
arrival_sec                 82860
Name: 14026, dtype: object

In [162]:
dep_time = time_to_normalized_sec('08:03:00')
for row in g.get_earliest_from(dep_time, g.conn_at_index(14026)):
    print(row)

line                             D
departure_time            08:07:00
arrival_time              08:09:00
start_stop         PL. GRUNWALDZKI
end_stop          most Grunwaldzki
start_stop_lat           51.111621
start_stop_lon           17.060086
end_stop_lat             51.110136
end_stop_lon             17.055093
departure_sec                29220
arrival_sec                  29340
Name: 24547, dtype: object
line                          111
departure_time           08:03:00
arrival_time             08:05:00
start_stop        PL. GRUNWALDZKI
end_stop                     Reja
start_stop_lat          51.111621
start_stop_lon          17.060086
end_stop_lat            51.112243
end_stop_lon            17.057503
departure_sec               28980
arrival_sec                 29100
Name: 510690, dtype: object


In [165]:
conn_lines = [g.get_earliest_from(dep_time, g.conn_at_index(idx)) for idx in pl_grunw_stops.index]

In [169]:
max_conn = max(conn_lines, key=lambda x: len(x))

In [172]:
max_conn

[line                          116
 departure_time           08:15:00
 arrival_time             08:18:00
 start_stop        PL. GRUNWALDZKI
 end_stop          PL. GRUNWALDZKI
 start_stop_lat          51.111981
 start_stop_lon          17.064034
 end_stop_lat            51.112057
 end_stop_lon            17.062886
 departure_sec               29700
 arrival_sec                 29880
 Name: 558565, dtype: object,
 line                          149
 departure_time           08:48:00
 arrival_time             08:50:00
 start_stop        PL. GRUNWALDZKI
 end_stop          PL. GRUNWALDZKI
 start_stop_lat          51.111981
 start_stop_lon          17.064034
 end_stop_lat            51.112189
 end_stop_lon            17.063358
 departure_sec               31680
 arrival_sec                 31800
 Name: 871107, dtype: object,
 line                                            253
 departure_time                             23:10:00
 arrival_time                               23:11:00
 start_stop

In [173]:
g.conn_graph[g.is_start_equal_to(pl_grunw_stop)][['end_stop', 'end_stop_lat', 'end_stop_lon']].drop_duplicates()

,end_stop,end_stop_lat,end_stop_lon
54000,Hala Stulecia,51.106895,17.07372


In [100]:
g.get_end_stop('PL. GRUNWALDZKI')

end_stop        PL. GRUNWALDZKI
end_stop_lat          51.111621
end_stop_lon          17.060086
dtype: object

In [88]:
dep_time = 8 * 3600

In [101]:
g.conn_at_index(71821)

line                                              2
departure_time                             08:02:00
arrival_time                               08:04:00
start_stop                          PL. GRUNWALDZKI
end_stop          Kliniki - Politechnika Wrocławska
start_stop_lat                            51.111452
start_stop_lon                            17.060529
end_stop_lat                              51.109206
end_stop_lon                              17.066414
departure_sec                                 28920
arrival_sec                                   29040
Name: 71821, dtype: object

In [133]:
g.conn_graph.loc[71820:71823]

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_sec,arrival_sec
71820,2,07:59:00,08:02:00,Reja,PL. GRUNWALDZKI,51.112786,17.055565,51.111452,17.060529,28740,28920
71821,2,08:02:00,08:04:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111452,17.060529,51.109206,17.066414,28920,29040
71822,2,08:04:00,08:06:00,Kliniki - Politechnika Wrocławska,Hala Stulecia,51.109206,17.066414,51.106895,17.073720,29040,29160
71823,2,08:06:00,08:07:00,Hala Stulecia,ZOO,51.106895,17.073720,51.105614,17.077418,29160,29220


In [155]:
for row in g.get_earliest_from(dep_time, g.conn_at_index(71820)):
    print(row)

line                                              2
departure_time                             08:02:00
arrival_time                               08:04:00
start_stop                          PL. GRUNWALDZKI
end_stop          Kliniki - Politechnika Wrocławska
start_stop_lat                            51.111452
start_stop_lon                            17.060529
end_stop_lat                              51.109206
end_stop_lon                              17.066414
departure_sec                                 28920
arrival_sec                                   29040
Name: 71821, dtype: object


In [143]:
g.get_neighbour_stops(pl_grunw_stop)

,stop,stop_lat,stop_lon
54000,Hala Stulecia,51.106895,17.07372


In [147]:
dep_time = time_to_normalized_sec('08:02:00')
for row in g.get_earliest_from(dep_time, g.conn_at_index(71820)):
    print(row)

line                                              2
departure_time                             08:02:00
arrival_time                               08:04:00
start_stop                          PL. GRUNWALDZKI
end_stop          Kliniki - Politechnika Wrocławska
start_stop_lat                            51.111452
start_stop_lon                            17.060529
end_stop_lat                              51.109206
end_stop_lon                              17.066414
departure_sec                                 28920
arrival_sec                                   29040
Name: 71821, dtype: object


In [104]:
g.conn_at_index(723854)

line                          131
departure_time           08:00:00
arrival_time             08:02:00
start_stop        PL. GRUNWALDZKI
end_stop                     Reja
start_stop_lat          51.111621
start_stop_lon          17.060086
end_stop_lat            51.112243
end_stop_lon            17.057503
departure_sec               28800
arrival_sec                 28920
Name: 723854, dtype: object

In [152]:
dep_time = time_to_normalized_sec('08:05:00')
g.get_earliest_from(dep_time, g.conn_at_index(723854))

[line                    111
 departure_time     08:05:00
 arrival_time       08:07:00
 start_stop             Reja
 end_stop            Katedra
 start_stop_lat    51.112243
 start_stop_lon    17.057503
 end_stop_lat      51.114199
 end_stop_lon      17.050798
 departure_sec         29100
 arrival_sec           29220
 Name: 510691, dtype: object,
 line                    253
 departure_time     23:41:00
 arrival_time       23:42:00
 start_stop             Reja
 end_stop            Katedra
 start_stop_lat    51.112243
 start_stop_lon    17.057503
 end_stop_lat      51.113306
 end_stop_lon      17.049847
 departure_sec         85260
 arrival_sec           85320
 Name: 918860, dtype: object]

In [58]:
# cost for the first node
g.sub_cost(dep_time, 71821, -1)

240

In [59]:
g.compute_stop_coords('Pomorska')

end_stop_lat    51.117895
end_stop_lon    17.030893
Name: 14126, dtype: object

In [60]:
g.compute_stop_coords('PL. GRUNWALDZKI')

end_stop_lat    51.111621
end_stop_lon    17.060086
Name: 14026, dtype: object

In [61]:
g.conn_at_index(14127)

line                      D
departure_time     08:43:00
arrival_time       08:44:00
start_stop         Pomorska
end_stop           Pomorska
start_stop_lat    51.117895
start_stop_lon    17.030893
end_stop_lat      51.117267
end_stop_lon      17.032001
departure_sec         31380
arrival_sec           31440
Name: 14127, dtype: object

In [62]:
g.conn_graph.loc[71821:71822]

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_sec,arrival_sec
71821,2,08:02:00,08:04:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,51.111452,17.060529,51.109206,17.066414,28920,29040
71822,2,08:04:00,08:06:00,Kliniki - Politechnika Wrocławska,Hala Stulecia,51.109206,17.066414,51.106895,17.073720,29040,29160


Od 8:00 dostaniemy się o 8:04, więc czas przejazdu - czyli koszt - to 240 sekund

In [175]:
goal_stop_coors = g.compute_stop_coords('Tramwajowa')

In [176]:
g.cost_getting_to(dep_time, 71821) + heuristic(g.conn_graph.loc[71821], goal_stop_coors)

1419.52

In [65]:
g.get_possible_stops('Pomorska').apply(g.stop_as_tuple, axis=1).values.tolist()

[('Pomorska', 51.11789543, 17.03089346),
 ('Pomorska', 51.11726735, 17.03200135),
 ('Pomorska', 51.117339, 17.031928),
 ('Pomorska', 51.1180115, 17.03074525),
 ('Pomorska', 51.11789231, 17.03100195),
 ('Pomorska', 51.11805751, 17.03056017),
 ('Pomorska', 51.11726746, 17.03189917),
 ('Pomorska', 51.11723359, 17.03175552)]

In [66]:
frontier = PriorityQueue()
frontier.put((10, 1))
frontier.put((5, 3))
print(frontier.get())
frontier.put((5, 3))
frontier.put((2, 8))
print(frontier.get())

(5, 3)
(2, 8)


In [67]:
frontier.get()

(5, 3)

Krotka jest w formacie (priority, data)

In [260]:
def print_info(conn, file=None):
    print(f"{conn['line']} goes from {conn['start_stop']} to {conn['end_stop']} and leaves at {conn['departure_time']}, arrives at {conn['arrival_time']}", 
          file=file)

In [261]:
def sec_to_time(seconds: int) -> str:
    seconds = int(seconds)
    hour = (seconds // 3600)
    minutes = (seconds % 3600) // 60 
    secs = (seconds % 3600) % 60 
    return f'{hour:02d}:{minutes:02d}:{secs:02d}'

In [262]:
def path_to_list(node: str, connections: dict):
    path = [node]
    while node:
        node = connections[node]
        path.append(node)
    path.reverse()
    return path[1:]

def idxs_to_nodes(graph: Graph, goal_idx: int, conn_idxs: dict):
    idx_path = path_to_list(goal_idx, conn_idxs)
    return [graph.conn_at_index(idx) for idx in idx_path[1:]]

def print_path(connections: dict):
    for conn in connections:
        print(f'{conn["start_stop"]} [{conn["departure_time"]}] --- {conn["line"]} ---> {conn["end_stop"]} [{conn["arrival_time"]}]')

In [ ]:
def run_solution(find_path_function, start_stop: str, goal_stop: str, leave_hour: str, change_time=0):
    start = timer()
    connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv', 
                               usecols=['line', 'departure_time', 'arrival_time', 'start_stop',
       'end_stop', 'start_stop_lat', 'start_stop_lon', 'end_stop_lat', 'end_stop_lon'])
    graph = Graph(connection_graph, add_constant_change_time)
    goal_index, came_from, costs = find_path_function(graph, start_stop, goal_stop, leave_hour)
    end = timer()
    elapsed_time = (end - start)
    # solution_cost = costs[graph.stop_as_tuple(graph.conn_at_index(goal_index))]
    return graph, goal_index, came_from, costs, elapsed_time 

In [ ]:
test_cases = pd.read_json(DATA_DIR / 'test_cases.json')
test_cases['dept_time'] = test_cases['dept_time'].apply(lambda x: x.strftime('%H:%M:%S'))

test_cases = test_cases.values

In [ ]:
def find_path(graph: Graph, start_stop: str, goal_stop: str, leave_hour: str):
    
    frontier = PriorityQueue()
    dep_time = time_to_normalized_sec(leave_hour)
    
    print(f'STOP {start_stop} --> TO {goal_stop}')

    cost_so_far = {}
    # if commuting A -> B, then this will be came_from[B] = A so we can recreate the path
    came_from = {}

    possible_goal_stops = graph.get_possible_stops(goal_stop).apply(graph.stop_as_tuple, axis=1).values.tolist()
    goal_stop_indexes = {}

    # given only stop name consider all possible start stops??  
    j = -1
    for _, candidate_start_stop in graph.get_possible_stops(start_stop).iterrows():
        candidate_stop_id = graph.stop_as_tuple(candidate_start_stop)
        cost_so_far[candidate_stop_id] = 0
        graph.add_conn(dep_time, candidate_start_stop, j)
        came_from[j] = None
        frontier.put((cost_so_far[candidate_stop_id], j))
        j -= 1

    with open(DATA_DIR / ('dijkstra_runs/' + re.sub(r"\W+", "", start_stop) + '-' + re.sub(r"\W+", "", goal_stop)), mode='w', encoding='utf-8') as f:

        i = 0
        while not frontier.empty():
            # get the stop with the lowest cost 
            cost, index = frontier.get()

            conn = graph.conn_at_index(index)
            current = graph.stop_as_tuple(graph.rename_stop(conn))

            # consider all possible end stops 
            if current[0] == goal_stop:
                goal_stop_indexes[current] = index
                if current not in possible_goal_stops:
                    continue
                possible_goal_stops.remove(current)
                if not possible_goal_stops:
                    break 

            print(f'[{i}]', file=f)
            for next_conn in graph.get_earliest_from(dep_time=dep_time + cost, start_stop=conn):
                print_info(next_conn, file=f)
                # cost of commuting start --> current and current --> next 
                new_cost = cost + graph.cost_getting_to(dep_time + cost, next_conn.name, conn.name)
                next_stop_id = graph.stop_as_tuple(graph.rename_stop(next_conn))
                if next_stop_id not in cost_so_far or new_cost < cost_so_far[next_stop_id]:
                    print(f'We arrive from {start_stop} to {next_stop_id} at {sec_to_time(dep_time + new_cost)}', file=f)
                    cost_so_far[next_stop_id] = new_cost
                    frontier.put((new_cost, next_conn.name))
                    came_from[next_conn.name] = index
            i+=1

    return goal_stop_indexes, came_from, cost_so_far

('Zagłębiowska', 51.07275516, 17.08914075)

In [268]:
g.get_possible_stops('Zagłębiowska')

,stop,stop_lat,stop_lon
620489,Zagłębiowska,51.074020,17.088013
620553,Zagłębiowska,51.072755,17.089141


In [270]:
g.get_neighbour_stops(g.rename_stop(g.conn_at_index(620489)))

,stop,stop_lat,stop_lon
620490,KSIĘŻE MAŁE,51.077843,17.083636


In [271]:
g.get_neighbour_stops(g.rename_stop(g.conn_at_index(620553)))

,stop,stop_lat,stop_lon
620554,Sosnowiecka,51.07046,17.091543


In [274]:
g.get_earliest_from(31240, g.conn_at_index(620553))

[line                       134
 departure_time        04:51:00
 arrival_time          04:52:00
 start_stop        Zagłębiowska
 end_stop           Sosnowiecka
 start_stop_lat       51.072755
 start_stop_lon       17.089141
 end_stop_lat          51.07046
 end_stop_lon         17.091543
 departure_sec            17460
 arrival_sec              17520
 Name: 759878, dtype: object]

In [272]:
graph, goal_stop_indexes, came_from, solution_cost, elapsed_time = run_solution(find_path, 'PL. GRUNWALDZKI', 'Kliniki - Politechnika Wrocławska', '08:00:00')

C:\Users\julia\AppData\Local\Temp\ipykernel_9516\3811924586.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


STOP PL. GRUNWALDZKI --> TO Kliniki - Politechnika Wrocławska


KeyError: 'line'

In [75]:
goal_stop_indexes

{('Kliniki - Politechnika Wrocławska', 51.10920637, 17.06641438): 71821,
 ('Kliniki - Politechnika Wrocławska', 51.109283, 17.066162): 551172,
 ('Kliniki - Politechnika Wrocławska', 51.10971261, 17.06511141): 60558,
 ('Kliniki - Politechnika Wrocławska', 51.10932241, 17.06628139): 832805}

In [76]:
solution_cost

{('PL. GRUNWALDZKI', 51.11162051, 17.06008642): 0,
 ('PL. GRUNWALDZKI', 51.11170028, 17.06043151): 0,
 ('PL. GRUNWALDZKI', 51.11114106, 17.0611933): 0,
 ('PL. GRUNWALDZKI', 51.1111569, 17.06146483): 0,
 ('PL. GRUNWALDZKI', 51.11162272, 17.05996513): 0,
 ('PL. GRUNWALDZKI', 51.11145165, 17.06052919): 0,
 ('PL. GRUNWALDZKI', 51.11144435, 17.05995961): 0,
 ('PL. GRUNWALDZKI', 51.11161063, 17.0604342): 0,
 ('PL. GRUNWALDZKI', 51.11245891, 17.0608282): 0,
 ('PL. GRUNWALDZKI', 51.11143827, 17.0603531): 0,
 ('PL. GRUNWALDZKI', 51.11198112, 17.06403361): 0,
 ('PL. GRUNWALDZKI', 51.11205745, 17.06288628): 0,
 ('PL. GRUNWALDZKI', 51.111406, 17.059852): 0,
 ('PL. GRUNWALDZKI', 51.11218871, 17.06335752): 0,
 ('PL. GRUNWALDZKI', 51.11195744, 17.06254965): 0,
 ('PL. GRUNWALDZKI', 51.11271353, 17.0650486): 0,
 ('Kochanowskiego', 51.11512731, 17.0737479): 180,
 ('most Grunwaldzki', 51.11013606, 17.05509252): 120,
 ('Kliniki - Politechnika Wrocławska', 51.109283, 17.066162): 120,
 ('Piastowska', 51.115

In [77]:
def get_add_cost_to_goal(goal_stop_ids, cost):
    for stop in goal_stop_ids:
        goal_stop_ids[stop] = (goal_stop_ids[stop], cost[stop])

In [78]:
get_add_cost_to_goal(goal_stop_indexes, solution_cost)

In [79]:
goal_stop_indexes

{('Kliniki - Politechnika Wrocławska', 51.10920637, 17.06641438): (71821, 120),
 ('Kliniki - Politechnika Wrocławska', 51.109283, 17.066162): (551172, 120),
 ('Kliniki - Politechnika Wrocławska', 51.10971261, 17.06511141): (60558,
  1200),
 ('Kliniki - Politechnika Wrocławska', 51.10932241, 17.06628139): (832805,
  2040)}

In [80]:
def get_cheapest_conn(goal_stop_index_cost):
    _, costs = zip(*list(goal_stop_index_cost.values()))
    min_cost = min(costs)
    return {stop: goal_stop_index_cost[stop] for stop in goal_stop_index_cost if goal_stop_index_cost[stop][1] == min_cost}

In [81]:
get_cheapest_conn(goal_stop_indexes)

{('Kliniki - Politechnika Wrocławska', 51.10920637, 17.06641438): (71821, 120),
 ('Kliniki - Politechnika Wrocławska', 51.109283, 17.066162): (551172, 120)}

In [82]:
def run_solution(find_path_function, start_stop: str, goal_stop: str, leave_hour: str, change_time=0):
    start = timer()
    connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv', 
                               usecols=['line', 'departure_time', 'arrival_time', 'start_stop',
       'end_stop', 'start_stop_lat', 'start_stop_lon', 'end_stop_lat', 'end_stop_lon'])
    graph = Graph(connection_graph, change_time)
    goal_indexes, came_from, costs = find_path_function(graph, start_stop, goal_stop, leave_hour)
    end = timer()
    elapsed_time = (end - start)

    get_add_cost_to_goal(goal_indexes, costs)
    cheepest_stops = get_cheapest_conn(goal_indexes)
    # solution_cost = costs[graph.stop_as_tuple(graph.conn_at_index(goal_index))]
    return graph, cheepest_stops, came_from, costs, elapsed_time 

In [83]:
graph, cheepest_stops, came_from, solution_cost, elapsed_time = run_solution(find_path, 'PL. GRUNWALDZKI', 'Kliniki - Politechnika Wrocławska', '08:00:00')

C:\Users\julia\AppData\Local\Temp\ipykernel_35176\2456682572.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


STOP PL. GRUNWALDZKI --> TO Kliniki - Politechnika Wrocławska


In [84]:
cheepest_stops

{('Kliniki - Politechnika Wrocławska', 51.10920637, 17.06641438): (71821, 120),
 ('Kliniki - Politechnika Wrocławska', 51.109283, 17.066162): (551172, 120)}

In [85]:
def dijkstra(start_stop: str, goal_stop: str, leave_hour: str, change_time=0):
    print(f'Commute from {start_stop} to {goal_stop} at {leave_hour}')
    graph, cheepest_stops, came_from, solution_cost, elapsed_time = run_solution(find_path, start_stop, goal_stop, leave_hour, change_time)
    
    connections = idxs_to_nodes(graph, goal_index, came_from)
    print_path(connections)
    print(f'Total trip time is {sec_to_time(solution_cost)}')
    commuting_time = sec_to_time(diff(connections[-1]['arrival_sec'], connections[0]['departure_sec']))
    print(f'Commuting takes {commuting_time}')
    print(f'Algorithm took {elapsed_time:.2f}s to execute')

In [125]:
for goal_stop in cheepest_stops:
    connections = idxs_to_nodes(graph, cheepest_stops[goal_stop][0], came_from)
    print_path(connections)

PL. GRUNWALDZKI [08:02:00] --- 2 ---> Kliniki - Politechnika Wrocławska [08:04:00]
PL. GRUNWALDZKI [08:00:00] --- 115 ---> Kliniki - Politechnika Wrocławska [08:02:00]


In [ ]:
for goal_stop in cheepest_stops:
    connections = idxs_to_nodes(graph, goal_stop[0], came_from)
    
print_path(connections)
print(f'Total trip time is {sec_to_time(solution_cost)}')
commuting_time = sec_to_time(diff(connections[-1]['arrival_sec'], connections[0]['departure_sec']))
print(f'Commuting takes {commuting_time}')
print(f'Algorithm took {elapsed_time:.2f}s to execute')

In [ ]:
came_from

{-1: None,
 -2: None,
 -3: None,
 -4: None,
 -5: None,
 -6: None,
 -7: None,
 -8: None,
 -9: None,
 -10: None,
 -11: None,
 -12: None,
 -13: None,
 -14: None,
 -15: None,
 -16: None,
 962747: -16,
 849563: -13,
 556876: -12,
 918837: -11,
 551172: -10,
 915296: -9,
 240333: -8,
 278072: -8,
 106752: -7,
 71821: -6,
 72378: -5,
 326389: -4,
 54062: -3,
 22596: -2,
 513060: -2,
 723854: -1,
 54063: 54062,
 72379: 72378,
 194986: 72378,
 278073: 326389,
 326390: 326389,
 551173: 551172,
 851073: 551172,
 510691: 723854,
 918860: 723854,
 14558: 849563,
 893890: 849563,
 22597: 22596,
 283: 14558,
 14559: 14558,
 54064: 54063,
 72380: 72379,
 326391: 326390,
 551174: 551173,
 932893: 551173,
 22598: 22597,
 932896: 22597,
 54065: 54064,
 106753: 106752,
 240334: 240333,
 551175: 551174,
 284: 283,
 459270: 283,
 534766: 283,
 849566: 283,
 14560: 14559,
 47739: 14559,
 497759: 14559,
 22599: 22598,
 556879: 22598,
 54066: 54065,
 66676: 72380,
 194988: 326391,
 168454: 326391,
 326392: 326

In [ ]:
solution_cost

{('PL. GRUNWALDZKI', 51.11162051, 17.06008642): 0,
 ('PL. GRUNWALDZKI', 51.11170028, 17.06043151): 0,
 ('PL. GRUNWALDZKI', 51.11114106, 17.0611933): 0,
 ('PL. GRUNWALDZKI', 51.1111569, 17.06146483): 0,
 ('PL. GRUNWALDZKI', 51.11162272, 17.05996513): 0,
 ('PL. GRUNWALDZKI', 51.11145165, 17.06052919): 0,
 ('PL. GRUNWALDZKI', 51.11144435, 17.05995961): 0,
 ('PL. GRUNWALDZKI', 51.11161063, 17.0604342): 0,
 ('PL. GRUNWALDZKI', 51.11245891, 17.0608282): 0,
 ('PL. GRUNWALDZKI', 51.11143827, 17.0603531): 0,
 ('PL. GRUNWALDZKI', 51.11198112, 17.06403361): 0,
 ('PL. GRUNWALDZKI', 51.11205745, 17.06288628): 0,
 ('PL. GRUNWALDZKI', 51.111406, 17.059852): 0,
 ('PL. GRUNWALDZKI', 51.11218871, 17.06335752): 0,
 ('PL. GRUNWALDZKI', 51.11195744, 17.06254965): 0,
 ('PL. GRUNWALDZKI', 51.11271353, 17.0650486): 0,
 ('Kochanowskiego', 51.11512731, 17.0737479): 180,
 ('most Grunwaldzki', 51.11013606, 17.05509252): 120,
 ('Kliniki - Politechnika Wrocławska', 51.109283, 17.066162): 120,
 ('Piastowska', 51.115

In [ ]:
solution_cost[('Kliniki - Politechnika Wrocławska', 51.109283, 17.066162)]

120

In [ ]:
came_from[goal_stop_indexes[('Kliniki - Politechnika Wrocławska', 51.109283, 17.066162)]]

-10

In [ ]:
def dijkstra(start_stop: str, goal_stop: str, leave_hour: str, change_time=0):
    print(f'Commute from {start_stop} to {goal_stop} at {leave_hour}')
    graph, goal_stop_indexes, came_from, solution_cost, elapsed_time = run_solution(find_path, start_stop, goal_stop, leave_hour, change_time)
    connections = idxs_to_nodes(graph, goal_index, came_from)
    print_path(connections)
    print(f'Total trip time is {sec_to_time(solution_cost)}')
    commuting_time = sec_to_time(diff(connections[-1]['arrival_sec'], connections[0]['departure_sec']))
    print(f'Commuting takes {commuting_time}')
    print(f'Algorithm took {elapsed_time:.2f}s to execute')

In [ ]:
dijkstra(*test_cases[5])

Commute from KOSZAROWA (Szpital) to Buforowa-Rondo at 02:35:00


C:\Users\julia\AppData\Local\Temp\ipykernel_22336\3766382681.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


STOP KOSZAROWA (Szpital) --> TO Buforowa-Rondo
KOSZAROWA (Szpital) [27:29:00] --- 246 ---> pl. Daniłowskiego [27:31:00]
pl. Daniłowskiego [27:31:00] --- 246 ---> Kasprowicza [27:32:00]
Kasprowicza [27:32:00] --- 246 ---> Berenta [27:33:00]
Berenta [27:33:00] --- 246 ---> KROMERA [27:35:00]
KROMERA [27:35:00] --- 246 ---> Mosty Warszawskie [27:37:00]
Mosty Warszawskie [27:37:00] --- 246 ---> Wyszyńskiego [27:38:00]
Wyszyńskiego [27:38:00] --- 246 ---> Ogród Botaniczny [27:40:00]
Ogród Botaniczny [27:40:00] --- 246 ---> Katedra [27:41:00]
Katedra [27:41:00] --- 246 ---> Urząd Wojewódzki (Muzeum Narodowe) [27:42:00]
Urząd Wojewódzki (Muzeum Narodowe) [27:42:00] --- 246 ---> Poczta Główna [27:43:00]
Poczta Główna [27:43:00] --- 241 ---> GALERIA DOMINIKAŃSKA [27:45:00]
GALERIA DOMINIKAŃSKA [27:45:00] --- 241 ---> Wzgórze Partyzantów [27:46:00]
Wzgórze Partyzantów [27:46:00] --- 241 ---> DWORZEC GŁÓWNY [27:47:00]
DWORZEC GŁÓWNY [27:47:00] --- 241 ---> DWORZEC GŁÓWNY [27:48:00]
DWORZEC GŁÓWNY

In [ ]:
def a_star_find_path(graph: Graph, start_stop: str, goal_stop: str, leave_hour: str):
   
    dep_time = time_to_normalized_sec(leave_hour)
    graph.add_start_conn(dep_time, start_stop)

    goal_stop_coords = graph.compute_stop_coords(goal_stop)

    cost = {}
    # if commuting A -> B, then this will be came_from[B] = A so we can recreate the path
    came_from = {}
    came_from[graph.get_start_idx()] = None
    cost[start_stop] = 0

    commute_so_far = {}
    commute_so_far[start_stop] = 0

    opened = [graph.get_start_idx()]
    closed = [] 

    def update(next_conn, current):
        next = next_conn['end_stop']
        commute_so_far[next] = commute_so_far[current] + graph.sub_cost(dep_time + commute_so_far[current], next_conn.name)
        cost[next] = commute_so_far[next] + heuristic(next_conn, goal_stop_coords)
        came_from[next_conn.name] = curr_conn.name

    with open(DATA_DIR / ('a_star_runs/' + re.sub(r"\W+", "", start_stop) + '-' + re.sub(r"\W+", "", goal_stop) + '.txt'), mode='w', encoding='utf-8') as f:
        
        i = 0
        while len(opened) > 0:
            print(f'[{i}]', file=f)
            curr_idx = None 
            curr_cost = None 

            # choose node with the lowest cost: what if we choose a connection 
            # that ends at the particular stop?
            for conn_idx in opened:
                candidate_stop = graph.conn_at_index(conn_idx)['end_stop']
                if curr_idx is None or cost[candidate_stop] < curr_cost:
                    curr_idx = conn_idx 
                    curr_cost = cost[candidate_stop]
                    
            curr_conn = graph.conn_at_index(curr_idx)
            current = curr_conn['end_stop']
            if current == goal_stop:
                goal_index = curr_conn.name 
                break 

            opened.remove(curr_idx)
            closed.append(curr_idx)
            
            for next_conn in graph.get_earliest_from(dep_time=dep_time + commute_so_far[current], start_stop=current):
                print_info(next_conn, file=f)
                next = next_conn['end_stop']
                if next_conn.name not in opened and next_conn.name not in closed:
                    opened.append(next_conn.name)
                    update(next_conn, current)
                    print(f"[NEW] We arrive from {start_stop} to {next} at {sec_to_time(dep_time + commute_so_far[next])}", file=f)
                else:
                    if commute_so_far[next] > commute_so_far[current] + graph.sub_cost(dep_time + commute_so_far[current], next_conn.name):
                        update(next_conn, current)
                        print(f"[UPDATED] We arrive from {start_stop} to {next} at {sec_to_time(dep_time + commute_so_far[next])}", file=f)
                        if next_conn.name in closed:
                            closed.remove(next_conn.name)
                            opened.append(next_conn.name)
            print(f'OPENED = {opened}', file=f)
            print(f'CLOSED = {closed}', file=f)
            print(f'Commute so far = {commute_so_far}', file=f)
            print(f'Cost = {cost}', file=f)
            print(f'--------------[{i}]------------------', file=f)
            i += 1
    
    return goal_index, came_from, cost


In [ ]:
graph, goal_index, came_from, solution_cost, elapsed_time = run_solution(a_star_find_path, 'Pomorska', 'Pomorska', '16:00:00')

In [ ]:
goal_index

In [ ]:
def a_star(start_stop: str, goal_stop: str, leave_hour: str, change_time=0):
    print(f'Commute from {start_stop} to {goal_stop} at {leave_hour}')
    graph, goal_index, came_from, solution_cost, elapsed_time = run_solution(a_star_find_path, start_stop, goal_stop, leave_hour, change_time)
    connections = idxs_to_nodes(graph, goal_index, came_from)
    print_path(connections)
    print(f'Solution cost is {solution_cost:.2f}')
    commuting_time = sec_to_time(diff(connections[-1]['arrival_sec'], connections[0]['departure_sec']))
    print(f"Total time is {sec_to_time(diff(connections[-1]['arrival_sec'], time_to_normalized_sec(leave_hour)))}")
    print(f'Commuting takes {commuting_time}')
    print(f'Algorithm took {elapsed_time:.2f}s to execute')

In [ ]:
a_star('Pomorska', 'Pomorska', '16:00:00')

In [ ]:
a_star(*test_cases[4])

In [ ]:
a_star(*test_cases[0])

In [ ]:
a_star(*test_cases[1])

In [ ]:
a_star(*test_cases[2])

In [ ]:
a_star(*test_cases[3])

In [238]:
graph, goal_index, came_from, solution_cost, elapsed_time = run_solution(find_path, *test_cases[4])

C:\Users\julia\AppData\Local\Temp\ipykernel_19180\3766382681.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


STOP PL. GRUNWALDZKI --> TO Renoma


In [239]:
goal_index

14560

In [240]:
came_from

{-1: None,
 -2: None,
 -3: None,
 -4: None,
 -5: None,
 -6: None,
 -7: None,
 -8: None,
 -9: None,
 -10: None,
 -11: None,
 -12: None,
 -13: None,
 -14: None,
 -15: None,
 -16: None,
 962747: -16,
 849563: -13,
 556876: -12,
 918837: -11,
 551172: -10,
 915296: -9,
 240333: -8,
 278072: -8,
 106752: -7,
 71821: -6,
 72378: -5,
 326389: -4,
 54062: -3,
 22596: -2,
 513060: -2,
 723854: -1,
 54063: 54062,
 72379: 72378,
 194986: 72378,
 278073: 326389,
 326390: 326389,
 551173: 551172,
 851073: 551172,
 510691: 723854,
 918860: 723854,
 14558: 849563,
 893890: 849563,
 22597: 22596,
 283: 14558,
 14559: 14558,
 54064: 54063,
 72380: 72379,
 326391: 326390,
 551174: 551173,
 932893: 551173,
 22598: 22597,
 932896: 22597,
 54065: 54064,
 106753: 106752,
 240334: 240333,
 551175: 551174,
 284: 283,
 459270: 283,
 534766: 283,
 849566: 283,
 14560: 14559,
 47739: 14559,
 497759: 14559,
 22599: 22598,
 556879: 22598,
 54066: 54065,
 66676: 72380,
 194988: 326391,
 168454: 326391,
 326392: 326

In [241]:
solution_cost

540

Przypadki testowe:


In [243]:
dijkstra(*test_cases[0])

Commute from KLECINA to OSIEDLE SOBIESKIEGO at 20:00:00


C:\Users\julia\AppData\Local\Temp\ipykernel_19180\3766382681.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


STOP KLECINA --> TO OSIEDLE SOBIESKIEGO
KLECINA [20:01:00] --- 107 ---> Skarbowców [20:02:00]
Skarbowców [20:02:00] --- 107 ---> Os. Przyjaźni [20:03:00]
Os. Przyjaźni [20:03:00] --- 107 ---> Zimowa [20:04:00]
Zimowa [20:10:00] --- D ---> KRZYKI [20:13:00]
KRZYKI [20:13:00] --- D ---> Orla [20:14:00]
Orla [20:14:00] --- D ---> Hallera [20:17:00]
Hallera [20:17:00] --- D ---> Rondo [20:19:00]
Rondo [20:19:00] --- D ---> Arkady (Capitol) [20:24:00]
Arkady (Capitol) [20:24:00] --- D ---> GALERIA DOMINIKAŃSKA [20:29:00]
GALERIA DOMINIKAŃSKA [20:29:00] --- D ---> Urząd Wojewódzki (Impart) [20:31:00]
Urząd Wojewódzki (Impart) [20:31:00] --- D ---> most Grunwaldzki [20:32:00]
most Grunwaldzki [20:32:00] --- D ---> PL. GRUNWALDZKI [20:34:00]
PL. GRUNWALDZKI [20:34:00] --- D ---> Kochanowskiego [20:37:00]
Kochanowskiego [20:37:00] --- D ---> Śniadeckich [20:38:00]
Śniadeckich [20:38:00] --- D ---> Zacisze [20:39:00]
Zacisze [20:39:00] --- D ---> Kwidzyńska [20:41:00]
Kwidzyńska [20:41:00] --- D

In [244]:
dijkstra(*test_cases[1])

Commute from Broniewskiego to Uniwersytet Ekonomiczny at 10:15:00


C:\Users\julia\AppData\Local\Temp\ipykernel_19180\3766382681.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


STOP Broniewskiego --> TO Uniwersytet Ekonomiczny
Broniewskiego [10:22:00] --- 908 ---> Trzebnicka [10:24:00]
Trzebnicka [10:24:00] --- 908 ---> DWORZEC NADODRZE [10:26:00]
DWORZEC NADODRZE [10:27:00] --- 144 ---> Paulińska [10:29:00]
Paulińska [10:29:00] --- 144 ---> Dubois [10:32:00]
Dubois [10:32:00] --- 144 ---> Pomorska [10:34:00]
Pomorska [10:34:00] --- 144 ---> Kępa Mieszczańska [10:36:00]
Kępa Mieszczańska [10:36:00] --- 144 ---> PL. JANA PAWŁA II [10:38:00]
PL. JANA PAWŁA II [10:38:00] --- 144 ---> pl. Orląt Lwowskich [10:40:00]
pl. Orląt Lwowskich [10:41:00] --- 15 ---> pl. Legionów [10:43:00]
pl. Legionów [10:43:00] --- 15 ---> Arkady (Capitol) [10:45:00]
Arkady (Capitol) [10:45:00] --- 15 ---> DWORZEC AUTOBUSOWY [10:48:00]
DWORZEC AUTOBUSOWY [10:48:00] --- 15 ---> Sanocka [10:50:00]
Sanocka [10:50:00] --- 15 ---> Uniwersytet Ekonomiczny [10:52:00]
Total trip time is 00:37:00
Commuting takes 00:30:00
Algorithm took 62.97s to execute


In [245]:
dijkstra(*test_cases[2])

Commute from POŚWIĘTNE to Młodych Techników at 15:30:00


C:\Users\julia\AppData\Local\Temp\ipykernel_19180\3766382681.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


STOP POŚWIĘTNE --> TO Młodych Techników
POŚWIĘTNE [15:34:00] --- 908 ---> Wołowska [15:35:00]
Wołowska [15:35:00] --- 908 ---> Kępińska [15:36:00]
Kępińska [15:36:00] --- 908 ---> Kamieńskiego [15:38:00]
Kamieńskiego [15:38:00] --- 908 ---> Broniewskiego [15:40:00]
Broniewskiego [15:41:00] --- 132 ---> Trzebnicka [15:43:00]
Trzebnicka [15:43:00] --- 132 ---> DWORZEC NADODRZE [15:45:00]
DWORZEC NADODRZE [15:45:00] --- 132 ---> Paulińska [15:47:00]
Paulińska [15:47:00] --- 132 ---> Dubois [15:49:00]
Dubois [15:49:00] --- 132 ---> Rynek [15:53:00]
Rynek [15:53:00] --- 132 ---> PL. JANA PAWŁA II [15:55:00]
PL. JANA PAWŁA II [15:55:00] --- 132 ---> Młodych Techników [15:57:00]
Total trip time is 00:27:00
Commuting takes 00:23:00
Algorithm took 12.73s to execute


Zmien czas by dawal czas przejazdu bez czekania na poczatku

In [246]:
dijkstra(*test_cases[3])

Commute from Śliczna to Marchewkowa at 23:50:00


C:\Users\julia\AppData\Local\Temp\ipykernel_19180\3766382681.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


STOP Śliczna --> TO Marchewkowa
Śliczna [23:55:00] --- K ---> Borowska (Aquapark) [23:56:00]
Borowska (Aquapark) [23:56:00] --- K ---> DWORZEC AUTOBUSOWY [23:58:00]
DWORZEC AUTOBUSOWY [23:58:00] --- K ---> DWORZEC GŁÓWNY [24:00:00]
DWORZEC GŁÓWNY [24:04:00] --- 243 ---> Wzgórze Partyzantów [24:06:00]
Wzgórze Partyzantów [24:06:00] --- 251 ---> GALERIA DOMINIKAŃSKA [24:08:00]
GALERIA DOMINIKAŃSKA [24:08:00] --- 251 ---> Hala Targowa [24:09:00]
Hala Targowa [24:09:00] --- 251 ---> Dubois [24:11:00]
Dubois [24:11:00] --- 251 ---> pl. Bema [24:13:00]
pl. Bema [24:13:00] --- 251 ---> Na Szańcach [24:13:00]
Na Szańcach [24:13:00] --- 251 ---> Jedności Narodowej [24:14:00]
Jedności Narodowej [24:14:00] --- 251 ---> Nowowiejska [24:16:00]
Nowowiejska [24:01:00] --- 242 ---> Daszyńskiego [24:02:00]
Daszyńskiego [24:02:00] --- 242 ---> Mosty Warszawskie [24:03:00]
Mosty Warszawskie [24:03:00] --- 242 ---> KROMERA [24:05:00]
KROMERA [24:05:00] --- 242 ---> Kromera (Czajkowskiego) [24:06:00]
Krome

In [99]:
graph, goal_index, came_from, solution_cost, elapsed_time = run_solution(find_path, *test_cases[4])

C:\Users\julia\AppData\Local\Temp\ipykernel_19180\2581666429.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv',


NameError: name 'print_info' is not defined

In [ ]:
came_from

In [ ]:
graph.conn_at_index(226197)

In [70]:
Graph(pd.read_csv(DATA_DIR / 'connection_graph.csv', 
                               usecols=['line', 'departure_time', 'arrival_time', 'start_stop', 'end_stop'], 
                               dtype=str)).get_stop_names()

array(['Zajezdnia Obornicka', 'Paprotna', 'Obornicka (Wołowska)',
       'Bezpieczna', 'Bałtycka', 'Broniewskiego', 'Pola', 'Syrokomli',
       'Kasprowicza', 'pl. Daniłowskiego', 'Przybyszewskiego',
       'Czajkowskiego', 'KOSZAROWA (Uniwersytet)', 'KOSZAROWA (Szpital)',
       'Berenta', 'KROMERA', 'Mosty Warszawskie', 'Wyszyńskiego',
       'Ogród Botaniczny', 'Katedra',
       'Urząd Wojewódzki (Muzeum Narodowe)', 'Poczta Główna',
       'skwer Krasińskiego', 'Wzgórze Partyzantów', 'Renoma',
       'Arkady (Capitol)', 'Pl. Hirszfelda', 'Krucza',
       'Krucza (Mielecka)', 'Inżynierska', 'Aleja Pracy', 'FAT',
       'GRABISZYŃSKA (Cmentarz)', 'Solskiego', 'Wiejska', 'Kadłubka',
       'Stanki', 'Bukowskiego', 'RACŁAWICKA', 'Rymarska', 'Wawrzyniaka',
       'Chłodna', 'Sowia', 'KRZYKI', 'Dworzec Główny (Dworcowa)',
       'Krasińskiego', 'Damrota', 'Koszarowa', 'Sołtysowicka',
       'Poprzeczna', 'Redycka', 'Bagatela', 'SOŁTYSOWICE', 'POŚWIĘTNE',
       'Wołowska', 'Kępińska', 'Ka